In [1]:
#sudo pip install ibm_db
import ibm_db
# conda install -c conda-forge ibm_db

import pandas as pd
import numpy as np
import psycopg2
# conda install -c anaconda psycopg2
import sqlalchemy
# conda install -c anaconda sqlalchemy
import matplotlib as plt
import requests
# conda install -c anaconda requests
from requests.auth import HTTPDigestAuth
import json
# conda install -c jmcmurray json
import urllib.request
import urllib.parse
# conda install -c anaconda zlib
import sys
import asyncio

from sqlalchemy import create_engine

import ibm_db_sa
# conda install -c auto ibm_db_sa
# pip install ibm-db-sa

import ibm_db_dbi

import logging

import datetime

from my_functions import *

In [2]:
logging.basicConfig(filename='logs.log',level=logging.DEBUG)


POSTGRES_ADDRESS = 'localhost'
POSTGRES_PORT = '5432'
POSTGRES_USERNAME = 'ebreshna_user'
POSTGRES_PASSWORD = 'secret'
POSTGRES_DBNAME = 'ebreshna_test'
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(username=POSTGRES_USERNAME, password=POSTGRES_PASSWORD, ipaddress=POSTGRES_ADDRESS, port=POSTGRES_PORT, dbname=POSTGRES_DBNAME))
# conn = psycopg2.connect("host=localhost dbname=asims_test user=asims_user password=secret port=5432")
# cur = conn.cursor()
psql_engine = create_engine(postgres_str)

# SQL SERVER CONNECTION
DB2_ADDRESS = '10.10.100.47'
DB2_PORT = '53000'
DB2_USERNAME = 'db2inst1'
DB2_PASSWORD = 'db2inst1'
DB2_DBNAME = 'mpower'
# db2_engine = create_engine('ibm_db_sa+pyodbc:///?odbc_connect={}'.format(quoted))
# db2_engine = create_engine("db2+ibm_db://user:pass@10.10.100.47:51000/mpower")
# conn = ibm_db.connect("DATABASE=mpower;HOSTNAME=host;PORT=50000;PROTOCOL=TCPIP;UID=username;PWD=password;", "", "")
db2_conn = ibm_db.connect("DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;", "", "")
# this is required to enable database access through Pandas
db2_conni =  ibm_db_dbi.connect(u"DRIVER={{IBM DB2 ODBC DRIVER}};DATABASE=mpower;HOSTNAME=10.10.100.47;PORT=53000;PROTOCOL=TCPIP;UID=db2inst1;PWD=db2inst1;")

In [3]:
# Region
cols_map = [
    {'src_col': 'ID', 'des_col': 'id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CATEGORY_ID', 'des_col': 'category_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'ENERGY_CHG', 'des_col': 'energy_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'MINIMUM_CHG', 'des_col': 'minimum_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'OTHER_CHG', 'des_col': 'other_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'CUSTOMER_CHG', 'des_col': 'customer_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'FIXED_CHG', 'des_col': 'fixed_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'MAX_LOAD_KW', 'des_col': 'max_load_kw', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CREATED_BY', 'des_col': 'created_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CREATE_DATE', 'des_col': 'create_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'UPDATED_BY', 'des_col': 'updated_by', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'UPDATE_DATE', 'des_col': 'update_date', 'src_dtype': 'TIMESTAMP', 'des_dtype': 'TIMESTAMP'},
    {'src_col': 'DB_IP_ADDRESS', 'des_col': 'db_ip_address', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SOURCE_OFFICE_ID', 'des_col': 'source_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'TARGET_OFFICE_ID', 'des_col': 'target_office_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'RECORD_STATUS', 'des_col': 'record_status', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SCREEN_ID', 'des_col': 'screen_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MONTH', 'des_col': 'month', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'YEAR', 'des_col': 'year', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'FLAG1', 'des_col': 'flag1', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG2', 'des_col': 'flag2', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG3', 'des_col': 'flag3', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG4', 'des_col': 'flag4', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'FLAG5', 'des_col': 'flag5', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'MODULE_ID', 'des_col': 'module_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'SUBMODULE_ID', 'des_col': 'submodule_id', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'CHECK_CONDITION', 'des_col': 'check_condition', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'IS_FLAT_RATE', 'des_col': 'is_flat_rate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'DEMAND_CHG', 'des_col': 'demand_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'CUST_IS_FLAT_RATE', 'des_col': 'cust_is_flat_rate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'REBATE', 'des_col': 'rebate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'ED', 'des_col': 'ed', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'MIN_IS_FLAT_RATE', 'des_col': 'min_is_flat_rate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'LOAD_UNIT', 'des_col': 'load_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'PHASE', 'des_col': 'phase', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'LPF_CHG', 'des_col': 'lpf_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'FIX_IS_FLAT_RATE', 'des_col': 'fix_is_flat_rate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'FIXED_MIN_CHG', 'des_col': 'fixed_min_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'DEMAND_MIN_CHG', 'des_col': 'demand_min_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'INT_ED', 'des_col': 'int_ed', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'FSA', 'des_col': 'fsa', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'CAP_SC', 'des_col': 'cap_sc', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'ADDL_DEMAND', 'des_col': 'addl_demand', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'LIGHTS_FANS', 'des_col': 'lights_fans', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'PER_LF_UNITS', 'des_col': 'per_lf_units', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'LATE_PAYMENT', 'des_col': 'late_payment', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'SUBSIDY_CHG', 'des_col': 'subsidy_chg', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'PFREBATE', 'des_col': 'pfrebate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'PFREBATE_FLAT', 'des_col': 'pfrebate_flat', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'LFREBATE', 'des_col': 'lfrebate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'LFREBATE_FLAT', 'des_col': 'lfrebate_flat', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'VOLT_REBATE', 'des_col': 'volt_rebate', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'VOLT_REBATE_FLAT', 'des_col': 'volt_rebate_flat', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'MIN_BILL_AMT', 'des_col': 'min_bill_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'MIN_BILL_UNIT', 'des_col': 'min_bill_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AVG_UNIT', 'des_col': 'avg_unit', 'src_dtype': 'DECIMAL', 'des_dtype': 'String'},
    {'src_col': 'AVG_BASE_UNIT', 'des_col': 'avg_base_unit', 'src_dtype': 'String', 'des_dtype': 'String'},
    {'src_col': 'AVG_AMT', 'des_col': 'avg_amt', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'CEIL_UNIT', 'des_col': 'ceil_unit', 'src_dtype': 'DECIMAL', 'des_dtype': 'numeric'},
    {'src_col': 'FIX_UNIT', 'des_col': 'fix_unit', 'src_dtype': 'DECIMAL', 'des_dtype': 'smallint'},
    {'src_col': 'TVM_25KW_HP', 'des_col': 'tvm_25kw_hp', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'AVERAGE_BILL', 'des_col': 'average_bill', 'src_dtype': 'String', 'des_dtype': 'boolean'},
    {'src_col': 'PROVIENCE', 'des_col': 'province', 'src_dtype': 'String', 'des_dtype': 'String'}
]

migrated_row_count = migrate_data(db2_conni, psql_engine, 'CENTRAL_ALL.LTTARIFF_PARA_T', 'public', 'lttariff_para_t', cols_map)
print("Total Migrated Records (lttariff_para_t): " + str(migrated_row_count))


Total Migrated Records (lttariff_para_t): Total Src Records: 96 Total migrated Records to dest: 96
